This notebook explores a few different potential models of the Milky Way

In [ ]:
using CairoMakie
using Revise
using LilGuys, Arya

In [ ]:
using PythonCall
agama = pyimport("agama")

In [ ]:
np = pyimport("numpy")

In [ ]:
using Printf

In [ ]:
include("agama_utils.jl")
include("agama_plots.jl")

In [ ]:
obs_props_filename = ENV["DWARFS_ROOT"] * "/observations/sculptor/observed_properties.toml"
icrs = LilGuys.coord_from_file(obs_props_filename)
icrs_err = LilGuys.coord_err_from_file(obs_props_filename)
gc_scl = LilGuys.transform(Galactocentric, icrs)

##  EP2021

In [ ]:
V_T2GYR

In [ ]:
lmc_coord = ICRS(ra = 80.8,
    dec = -69.8, 
    pmra = 1.910,
    pmdec = 0.229,
    radial_velocity = 262,
    distance = 49.59
    )

lmc_coord_err = ICRS(
    ra=0.3,
    dec=0.3,
    distance=0.54,
    pmra=0.2,
    pmdec=0.47,
    radial_velocity=3,
    )

In [ ]:
gc = LilGuys.transform(Galactocentric, lmc_coord)

In [ ]:
ep20 = load_agama_potential("EP2020.ini")
galpy14 = load_agama_potential("galpy_2014.ini") 

In [ ]:
pot = ep20

In [ ]:
σv = calc_σv_interp(pot)

In [ ]:
lines(log10.(σv.x), σv.y * V2KMS)

In [ ]:
orbit_nofric = calc_orbit(gc, pot, time=-10/T2GYR)

In [ ]:
Mmw = pyconvert(Float64, pot.enclosedMass(300))

In [ ]:
Mlmc = 10.

In [ ]:
f_fric(pos, vel) = a_dyn_friction(pos, vel, r_s=10, σv=σv, ρ=x->pyconvert(Float64, pot.density(x)), M=Mlmc)

In [ ]:
use_noninacc = 1

In [ ]:
f_grav(pos, vel) = (1 + use_noninacc * Mlmc/Mmw) * pyconvert(Vector{Float64}, pot.force(pos)) 

In [ ]:
f_acc(pos, vel) =  f_grav(pos, vel) .+ f_fric(pos, vel)

In [ ]:
orbit_2 = leap_frog(gc, f_acc, dt_max=1)

In [ ]:
fig, ax, _ = plot_r_t(orbit_nofric)
plot_r_t!(ax, orbit_2)

fig

In [ ]:
fig, ax, p = plot_y_z(orbit_nofric)
lines!(orbit_2.position[2, :], orbit_2.position[3, :])

fig


## Adding a galaxy...

In [ ]:
ep20 = load_agama_potential("EP2020.ini")


In [ ]:
pot = load_agama_potential("EP2020.ini")

In [ ]:
orbit = calc_orbit(gc, ep20, units=:code, time=-10)

In [ ]:
plot_r_t(orbit)

In [ ]:
plot_y_z(orbit)

In [ ]:
plot_v_circ(ep20)

In [ ]:
mm11 = load_agama_potential("mcmillan11.ini")
mm17 = load_agama_potential("mcmillan17.ini")
bt08 = load_agama_potential("BT08.ini")
piffl14 = load_agama_potential("piffl_14.ini")
J95 = load_agama_potential("J95.ini")
galpy14 = load_agama_potential("galpy_2014.ini")

In [ ]:
potentials_light = [
    "EP20" => ep20,
    "bt08" => bt08,
    "galpy14" => galpy14,
    ]


In [ ]:
potentials_heavy = [
    "EP20" => ep20,
    "mcmillan 11" => mm11,
    "piffl14" => piffl14,
    "J95" => J95,

]

In [ ]:
potentials = [
    "EP20" => ep20,
    "mcmillan 17" => mm17,

]

In [ ]:
my_potentials = {
    "fiducial": EP2020,
    "light": light_mw,
    "heavy": heavy_mw,
    "flat": me_flat,
    "steep": me_steep
}

In [ ]:
for label, pot in my_potentials.items():
    print(label, o.jr(pot))

In [ ]:
o.jr(galpy14)

In [ ]:
o.jr(galpy14_heavy)

In [ ]:
o.jr(EP2020)

In [ ]:
o.jr(ablimit20a)

In [ ]:
o.jr(mcmillan2017)

In [ ]:
for label, pot in my_potentials.items():
    print(label, o.jz(pot))

In [ ]:
o.jz(galpy14)

In [ ]:
o.jz(galpy14_heavy)

In [ ]:
o.jz(EP2020)

In [ ]:
o.jz(mcmillan2017)

In [ ]:
plot_Vc(EP2020_bulge)
plot_Vc(mcmillan2011_bulge)
plot_Vc(galpy14_bulge)
plot_Vc(pouliasis17_bulge)

In [ ]:
plot_Vc(EP2020_thick + EP2020_thick)
plot_Vc(galpy14_disk)
# plot_Vc(galpy14_disk)

plot_Vc(pouliasis17_thick + pouliasis17_thick)
# plot_Vc(pouliasis17b_thick + pouliasis17b_thick)

In [ ]:
plot_Vc(EP2020_halo)
plot_Vc(mcmillan2011_halo)
plot_Vc(mcmillan2017_halo)
plot_Vc(galpy14_halo)
plot_Vc(2*galpy14_halo)

plot_Vc(pouliasis17_halo)
plot_Vc(pouliasis17b_halo)
plot_Vc(ablimit_b_comp["halo"]) # pretty much same as galpy


In [ ]:
dwarf_galaxy_gc.icrs

In [ ]:
o = Orbit(dwarf_galaxy_gc)

In [ ]:
pot = EP2020
ts = np.linspace(0, -5, 10_000) * u.Gyr
o.integrate(ts, pot)

In [ ]:
fig, ax = plt.subplots()

plt.plot(o.R(ts), o.z(ts))
ax.set_aspect(1)

In [ ]:
fig, ax = plt.subplots()

plt.plot(o.y(ts), o.z(ts))

ax.set_aspect(1)

In [ ]:
plt.plot(ts, o.r(ts))

In [ ]:
pos_gc = []

ts = np.linspace(0, 0.1, 10_000) * u.Gyr
o.integrate(ts, pot)

# galpy uses left-handed frame :/
pos_gc = coord.SkyCoord(
    x=- o.x(ts), y=o.y(ts), z=o.z(ts), 
    v_x = -o.vx(ts), v_y=o.vy(ts), v_z=o.vz(ts),
    frame=gc_frame)

pos_icrs = pos_gc.transform_to("icrs")

In [ ]:
plt.scatter(pos_icrs.ra, pos_icrs.dec, c=ts)
plt.colorbar()

plt.plot(o.ra(ts), o.dec(ts))

plt.scatter(dwarf_galaxy.ra, dwarf_galaxy.dec)


tpm = 3e7*u.yr
plt.plot(sculptor.ra + sculptor.pm_ra_cosdec / np.cos(sculptor.dec) * tpm * np.array([0, 1]),
        sculptor.dec + sculptor.pm_dec * tpm * np.array([0,1]),
         zorder=5, color=arya.COLORS[2]
        )

plt.xlabel("RA")
plt.ylabel("dec")

In [ ]:
sc2 = dwarf_galaxy_gc.transform_to(coord.ICRS)

# Alla Vasiliev+++

In [ ]:
using CSV, DataFrames

In [ ]:
vasiliev21_frozen = load_agama_potential("vasiliev+21/potential_nolmc.ini")
vasiliev21 = load_agama_potential("vasiliev+21/potential_evolving.ini")
vasiliev21_lmc = load_agama_potential("vasiliev+21/potential_lmc_only.ini")

In [ ]:
# loads in trajectory of lmc in Vasiliev 2021
lmc_file = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev+21/trajlmc.txt"
lmc_traj = CSV.read(lmc_file, DataFrame, delim=" ", header = [:time, :x, :y, :z, :v_x, :v_y, :v_z])

filt = lmc_traj.time .<= 0
lmc_traj = lmc_traj[filt, :]
lmc_traj = DataFrame(reverse(eachrow(lmc_traj)))


orbit_lmc_act = Orbit(time=lmc_traj.time  * V_T2GYR/ T2GYR, 
    position = [lmc_traj.x lmc_traj.y lmc_traj.z]',
    velocity = [lmc_traj.v_x lmc_traj.v_y lmc_traj.v_z]' * V_V2KMS / V2KMS
    )



In [ ]:
vasiliev21_lmc.enclosedMass(10, 5)

In [ ]:
function enclosedMass(vasiliev21, r=60; time=0., direction=[1, 1, 1])
    direction = direction / calc_r(direction)
    a = pyconvert(Vector{Float64}, vasiliev21_lmc.force(r * direction, t=time))
    a = calc_r(a)
    return a * r^2
end
    

In [ ]:
py2f(x) = pyconvert(Float64, x)
py2vec(x) = pyconvert(Vector{Float64}, x)

In [ ]:
lmc_coord = ICRS(ra = 81,
    dec = -69.75, 
    pmra = 1.8,
    pmdec = 0.35,
    radial_velocity = 260,
    distance = 50.
    )

In [ ]:
lmc_gc = LilGuys.transform(LilGuys.Galactocentric, lmc_coord)

In [ ]:
function make_lmc_pot(Mlmc, r_s; kwargs...)
    Φ_lmc = agama.Potential(type="Spheroid", alpha=1, beta=3, gamma=1, 
            scaleradius = r_s,
            mass = Mlmc, 
            outercutoffradius = 10r_s;
            kwargs...
        )
    return Φ_lmc
end


In [ ]:
function calc_lmc_orbit(pot, gc_coord; 
        Mlmc = 15,
        reflex_motion = true,
        dynamical_friction = true,
        Λ = nothing,
        σv = nothing,
        vasiliev_units = false,
        time = -5 / T2GYR, 
        r_s = 8.5 * (Mlmc / 10)^0.6,
        timestep = abs(time) / 1e4
    )

    
    if vasiliev_units
        acc_scale = (V_V2KMS/ V2KMS)^2  * (V_R2KPC / R2KPC)^-1
        v_scale = V2KMS / V_V2KMS
        m_scale = M2MSUN / V_M2MSUN
    else
        v_scale = m_scale = acc_scale = 1
    end
    

    if σv === nothing
        calc_σv_interp(pot, log_r=LinRange(2, -2.0, 1000))
    end

    ρ(x) = py2f(pot.density(x))
    
    Φ_lmc = make_lmc_pot(Mlmc * m_scale, r_s)

    f_fric(pos, vel) = dynamical_friction * a_dyn_friction(pos, vel, 
        r_s=r_s * 1.3,
        σv=σv, ρ=ρ, M=Mlmc * m_scale, Λ = Λ
    )


    f_grav(pos, vel) = py2vec(
        pot.force(pos) - reflex_motion * Φ_lmc.force(-pos)
    )

    f_acc(pos, vel) = acc_scale * (f_grav(pos, vel*v_scale) .+ f_fric(pos, vel *v_scale) )

    orbit = leap_frog(gc_coord, f_acc, time=time, timestep=timestep)

    return orbit
end


In [ ]:
function make_lmc_mw_orbit_from_orbit(pot, orbit;
        reflex_motion=true,
        Mlmc=15,
        r_s=8.5 * (Mlmc / 10)^0.6, 
        vasiliev_units=false, 
        kwargs...
    
    )
    time = orbit.time

    if vasiliev_units
        Mlmc *= M2MSUN / V_M2MSUN
        time *= T2GYR / V_T2GYR
    end
    
    if orbit.time[2] < orbit.time[1]
        position = orbit.position[:, end:-1:1]
        time = reverse(time)
    end

    centre = vcat(time', position)

    centre = PyArray(centre')

    pot_lmc = make_lmc_pot(Mlmc, r_s, center=centre)

    
    if reflex_motion
        N = length(time)
        a_reflex = pot_lmc.force(zeros(length(time), 3), t=time)
        nptime = PyMatrix(reshape(time,(:, 1)))
        mat_reflex = np.hstack([nptime, -a_reflex])        
        pot_reflex = agama.Potential(type="UniformAcceleration", file=mat_reflex)
        
        Φ = agama.Potential(pot, pot_lmc, pot_reflex)
    else
        Φ = agama.Potential(pot, pot_lmc)
    end

    return Φ
end

In [ ]:
function make_lmc_mw_orbit(pot, lmc_gc;
        reflex_motion=true,
        Mlmc=15,
        r_s=8.5 * (Mlmc / 10)^0.6, 
        vasiliev_units=false, 
        kwargs...
    
    )
    orbit = calc_lmc_orbit(pot, lmc_gc; Mlmc=Mlmc, r_s=r_s, vasiliev_units=vasiliev_units, reflex_motion=reflex_motion, kwargs...)
    time = orbit.time
    

    if vasiliev_units
        Mlmc *= M2MSUN / V_M2MSUN
        time *= T2GYR / V_T2GYR
    end
    
    
    if orbit.time[2] < orbit.time[1]
        position = orbit.position[:, end:-1:1]
        time = reverse(time)
    end

    centre = vcat(time', position)

    println(centre[:, 1:4])
    centre = PyArray(centre')

    pot_lmc = make_lmc_pot(Mlmc, r_s, center=centre)

    
    if reflex_motion
        N = length(time)
        a_reflex = pot_lmc.force(zeros(length(time), 3), t=time)
        nptime = PyMatrix(reshape(time,(:, 1)))
        mat_reflex = np.hstack([nptime, -a_reflex])
        println(a_reflex)
        
        pot_reflex = agama.Potential(type="UniformAcceleration", file=mat_reflex)
        
        Φ = agama.Potential(pot, pot_lmc, pot_reflex)
    else
        Φ = agama.Potential(pot, pot_lmc)
    end

    return Φ, orbit
end

In [ ]:
Φ_v21_simple = make_lmc_mw_orbit_from_orbit(vasiliev21_frozen, orbit_lmc_act,
    Mlmc=15, 
    vasiliev_units=true,
    reflex_motion=true,
    dynamical_friction=true,
)

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_orbit(vasiliev21_frozen, lmc_gc,
    Mlmc=15, 
    σv=x->120, 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=true
)

In [ ]:
orbit_lmc

In [ ]:
orbit_scl = calc_orbit(gc_scl, Φ_evolving, time=-5, units=:vasiliev, N=10_001)

In [ ]:
orbit_scl_v21_simple = calc_orbit(gc_scl, Φ_v21_simple, time=-5, units=:vasiliev, N=10_001)

In [ ]:
orbit_scl_nolmc = calc_orbit(gc_scl, vasiliev21_frozen, time=-5, units=:vasiliev)

In [ ]:
v21_noref = agama.Potential(vasiliev21[0], vasiliev21[1], vasiliev21[3], vasiliev21[4])

In [ ]:
orbit_scl_act = calc_orbit(gc_scl, vasiliev21, time=-5, units=:vasiliev, N=10001)

In [ ]:
orbit_lmc_act = resample(orbit_lmc_act, orbit_scl.time)

In [ ]:
orbit_scl_lmc  = orbit_scl - orbit_lmc 
orbit_scl_lmc_act = orbit_scl_act - orbit_lmc_act
orbit_scl_lmc_simple = orbit_scl_v21_simple - orbit_lmc_act

In [ ]:
plot_y_z([
    "vasiliev+21" => orbit_scl_lmc_act,
    "vasiliev+21 simple" => orbit_scl_lmc_simple,
        "approx" => orbit_scl_lmc,

        ])

In [ ]:
plot_y_z([
    "lmc me" => orbit_lmc,
    "lmc vasiliev+21" => orbit_lmc_act,
    "scl me" => orbit_scl,
    "scl vasiliev+21" => orbit_scl_act,
    "scl vasiliev+21 simple" => orbit_scl_v21_simple,

        ])

In [ ]:
lmc_me = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=true, reflex_motion=true,
    timestep=:adaptive
)

lmc_point = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=false, reflex_motion=false
)

lmc_dyf = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=true, reflex_motion=false
)


lmc_ref = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=false, reflex_motion=true
)

In [ ]:
orbits = [
    "LMC-MW" => lmc_orbit,
    "point" => lmc_point,
    "reflex" => lmc_ref,
    "dyn fric" => lmc_dyf,
    "lmc me" => lmc_me,
    "lmc2" => orbit_lmc,
    ];

In [ ]:
plot_y_z(orbits)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])
plot_v_circ!(Φ_lmc, vasiliev_units=true, log=false, log_r=log10.(LinRange(0.1, 50, 100)))

fig

In [ ]:
Φ_lmc.density([30, 0, 0])

In [ ]:
LilGuys.Plots.plot_xyz(lmc_orbit.position,orbit_lmc_me_2.position, labels=["LMC", "Scl", "Scl (no lmc)"])

In [ ]:
LilGuys.Plots.plot_xyz(lmc_orbit.position, scl_lmc_orbit.position, scl_orbit.position, labels=["LMC", "Scl", "Scl (no lmc)"])

In [ ]:
fig, ax, p = plot_y_z(scl_m_lmc)
Colorbar(fig[1, 2], p)
fig

In [ ]:
scl_orbit = calc_orbit(gc, vasiliev21_frozen, units=:vasiliev, time=-5)

In [ ]:
scl_m_lmc = deepcopy(scl_lmc_orbit)

scl_m_lmc.position .-= lmc_orbit.position

In [ ]:
scl_lmc_orbit = calc_orbit(gc, vasiliev21, units=:vasiliev, time=lmc_traj.time) # need to pass vasiliev time to this class...

In [ ]:
orbits = [
    "Scl-MW" => scl_lmc_orbit,
    "Scl-MW (no LMC)" => scl_orbit,
    "LMC-MW" => lmc_orbit,
   # "lmc me" => orbit_lmc_me,
    ]

In [ ]:
plot_y_z(orbits)

In [ ]:
r_scl_lmc = calc_r(lmc_orbit.position, scl_lmc_orbit.position)

In [ ]:
R2KPC

In [ ]:
V_R2KPC

In [ ]:
V_M2MSUN / M2MSUN 

In [ ]:
T2GYR / V_T2GYR

In [ ]:
V_V2KMS / V2KMS

In [ ]:
fig = plot_r_t(orbits, legend=false)

lines!(lmc_orbit.time * T2GYR, r_scl_lmc, label="Scl-LMC ")
axislegend()
fig

In [ ]:
log10(LilGuys.kpc_to_arcmin(LilGuys.calc_break_radius(8.0 / V2KMS, 0.1 / T2GYR), 83.2))

In [ ]:
log10(20)

In [ ]:
fig, ax = FigAxis(
    xlabel = "time / Gyr",
    ylabel = "distance from Scl to LMC / kpc",
    limits=(nothing, nothing, 0, nothing),
    )

lines!(lmc_orbit.time * T2GYR, r_scl_lmc)

fig

In [ ]:
lmc_orbit.time[argmin(r_scl_lmc)] * T2GYR

In [ ]:
minimum(r_scl_lmc)

In [ ]:
lmc_orbit.time ./ scl_lmc_orbit.time

In [ ]:
plot_v_circ(ep20)

In [ ]:
plot_v_circ(vasiliev21, vasiliev_units=true)

# MC orbits

In [ ]:
include("agama_utils.jl")

In [ ]:
icrs_samples = LilGuys.rand_coords(icrs, icrs_err, 1000);
gc_samples = LilGuys.transform.(Galactocentric, icrs_samples);

In [ ]:
icrs_df = LilGuys.to_frame(icrs_samples)

In [ ]:
ic = make_agama_init(gc_samples)

In [ ]:
lmc_orbit.time

lmc.times

In [ ]:
orbits = calc_orbits(gc_samples, ep20, time=-5 / T2GYR);

In [ ]:
plot_y_z(orbits[1].second)

In [ ]:
plot_r_t(orbits[1:100], alpha=0.2, color=:black, legend=false)

In [ ]:
orbits = calc_orbits(gc_samples, vasiliev21_frozen, time=-5 / V_T2GYR, units=:vasiliev);

In [ ]:
plot_y_z(orbits[1].second)

In [ ]:
plot_r_t(orbits[1:100], alpha=0.2, color=:black, legend=false)

In [ ]:
orbits = calc_orbits(gc_samples, vasiliev21, time=lmc_traj.time, units=:vasiliev);

In [ ]:
plot_r_t(orbits[1:100], alpha=0.2, color=:black, legend=false)

In [ ]:
rs = [calc_r(lmc_orbit.position, o.position) for (l, o) in orbits];

In [ ]:
rs2 = [calc_r(orbit_lmc_me.position, o.position) for (l, o) in orbits];

In [ ]:
scl_lmc_times = lmc_orbit.time[argmin.(rs)] * T2GYR

In [ ]:
scl_lmc_mins = minimum.(rs)

In [ ]:
hist(scl_lmc_times)

In [ ]:
hist(scl_lmc_mins)

In [ ]:
scatter(icrs_df.pmra, scl_lmc_mins)

In [ ]:
scatter(icrs_df.pmdec, scl_lmc_mins)

In [ ]:
scatter(icrs_df.distance, scl_lmc_mins)

In [ ]:
scatter(icrs_df.distance, scl_lmc_times)

In [ ]:
fig, ax = FigAxis(
    xlabel = "time / Gyr",
    ylabel = "distance from Scl to LMC / kpc",
    limits=(nothing, nothing, 0, nothing),
    )


for r in rs
    lines!(lmc_orbit.time * T2GYR, r, alpha=0.05, color=:black)
end

fig

In [ ]:
orbits[1].second.position

In [ ]:
fig, ax = FigAxis(
    xlabel = "time / Gyr",
    ylabel = "distance  / kpc",
    limits=(nothing, nothing, 0, nothing),
    )


lines!(lmc_orbit.time * T2GYR, rs[1], label="Scl - LMC")
lines!(lmc_orbit.time * T2GYR, rs2[1], label="Scl - LMC point particle")

lines!(orbits[1].second.time * T2GYR, calc_r(orbits[1].second.position), label="Scl - MW")

axislegend(position=:lb)

fig